### 9.5.1 Set Up the Database and Flask

In [23]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

#### Set Up the Database

In [24]:
# Access the SQLite database
engine = create_engine("sqlite:///hawaii.sqlite")

# Reflect the database into our classes
Base = automap_base()

# Reflect our tables
Base.prepare(engine, reflect=True)

# Save our references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

# Finally, create a session link from Python to our database
session = Session(engine)

#### Set Up Flask

In [25]:
app = Flask(__name__)

In [26]:
# Define the welcome route
@app.route("/")

# Add the routing information for each of the other routes. 
# For this create a function and the return statement will have f-strings as a reference to all of the other routes. 
# /api/v1.0/ means this is version 1 of our application


def welcome():
    return(
    '''
    Welcome to the Climate Analysis API!
    Available Routes:
    /api/v1.0/precipitation
    /api/v1.0/stations
    /api/v1.0/tobs
    /api/v1.0/temp/start/end
    ''')

#### 9.5.3 Precipitation Route

In [27]:
# Create a dictionary with the date as the key and the precipitation as the value at the end of the function
# Jsonify() is a function that converts the dictionary to a JSON file.

@app.route("/api/v1.0/precipitation")
def precipitation():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(Days=365)
    precipitation = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= prev_year).all()
    precip = {date: prcp for date, prcp in precipitation}
    return jsonify(precip)

#### 9.5.4 Stations Route

In [28]:
# Unraveling our results into a one-dimensional array using the function np.ravel(), with results as our parameter
# To convert the results to a list use the list function - list(). 
# Then jsonify the list and return it as JSON

@app.route("/api/vi.0/stations")
def stations(): 
    results = session.query(Station.station).all()
    stations = list(np.ravel(results))
    return jsonify(stations=stations)


#### 9.5.5 Monthly Temperature Route

In [29]:
@app.route("/api/v1.0/tobs")
def temp_monthly():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    results = session.query(Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').\
        filter(Measurement.date >= prev_year).all()
    temps = list(np.unravel(results))
    return jsonify(temps=temps)

#### 9.5.6 Statistics Route

In [31]:
@app.route("/api/v1.0/temp/<start>")
@app.route("/api/v1.0/temp/<start>/<end>")
def stats(start=None, end=None):
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    
    if not end:
        results = session.query(*sel).\
            filter(Measurement.date >= start).all()
        temps = list(np.ravel(results))
        
        results = session.query(*sel).\
            filter = (Measurement.date >= start).\
            filter = (Measurement.date <= end).all()
        temps = list (np.ravel(results))
    return jsonify(temps=temps)